In [1]:
#======================================================================================================
# by kobongsoo, 
# ============= HISTORY =============================================
# make by: 2022-04-13
# updata: 2022-09-27
# => KlueNLI 데이터 셋 추가
# => 데이터 셋은 KorNLI(550,152) 가 KlueNLI(24,998)보다 20배정도 크지만, 
# Klue 데이터 셋이 실제 모델 훈련에는 더 좋다고 함.
# ======================================================================
#
# sentence-bert NLI 훈련 및 평가 예시
# => 기존 (distil)bert 모델을 가지고, NLI로 훈련 및 평가 후, S-BERT로 만드는 예시임.
#
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄일수 있음
#=> reduce_out_dimension = True 로 하면, 출력 임베딩 dimension이 줄어들게 설정가능함
#
# => sentence-transformers 패키지를 이용하여 구현 함.(*pip install -U sentence-transformers 설치 필요)
#
# 도큐먼트 : https://www.sbert.net/index.html
# 소스참고 : https://github.com/BM-K/KoSentenceBERT-ETRI

# pip install -U sentence-transformers
#======================================================================================================
import torch.nn as nn
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import sys
import os
import gzip
import csv

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert", logfilename="../../log/s-bert")
device = GPU_info()

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


logfilepath:../../log/s-bert_2022-10-25.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#s-bert로 만들 원본 bert 경로
#model_path = "../../data11/model/sbert/sbert-mdistilbertV3.1.1-disitl-ns-1" 
model_path = 'klue/bert-base'

#원본 bert를 sentencebert로 만든후 만들어진 s-bert 저장 경로
#=> **해당 경로\eval 폴더에 similarity_evaluation_sts-dev_result.csv 파일로 각 epoch 마다 평가된 결과가 기록된다.
#smodel_path = 'output/training_nli_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S") 
smodel_path = "../../data11/model/NLI/sbert-klue-bert-base-gluenli"

use_kornli = 0 # kornli 파일 
use_kluenli = 0 # kluests_v1.1 파일 
use_gluenli = 1 # glue 파일

#KorNLI, KorSTS 파일 경로
train_kornli_file = '../../data11/korpora/kornli/snli_1.0_train.ko.tsv' 
eval_korsts_file = '../../data11/korpora/korsts/tune_dev.tsv'

#KLUENIL, KlueSTS 파일 경로
train_kluenli_file = '../../data11/korpora/klue-nli/klue-nli-v1.1_train.json' 
eval_kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

#GLUENLI, GLUESTS 파일 경로
train_gluenli_file = '../../data11/korpora/gluemnli/glue-mnli-train.tsv' 
eval_gluests_file = '../../data11/korpora/gluestsb/glue-stsb-valid.tsv'

#============================================================================ 
train_batch_size = 32 
eval_batch_size = 64 
num_epochs = 3 # 8 정도면 최상의 값 찾을수 있음 
max_seq_length = 72 
lr = 3e-5 # default=2e-5 
eps = 1e-8 #lr이 0으로 나뉘어져 계산이 엉키는 것을 방지하기 위해 epsilion
seed = 111 
#============================================================================
#* True=모델에 상관없이 영어는 소문자로 입력하겠다는 뜻(glue는 소문자로만 비교하는게 더 적확함), 한국어는 True도 상관없음
#do_lower_case = True 
do_lower_case = False
#============================================================================
#*출력 dimension을 줄일 경우에는 True로 하고, out_dimension에 줄일 값을 설정함
reduce_out_dimension = False # True이면 dimension을 줄임=>Dense 모델 추가됨 
out_dimension = 128 
#============================================================================

seed_everything(seed)

#모델과 tokenizer 를 불러옴
#=> **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=do_lower_case)

#word embedding_model 출력
print(word_embedding_model)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Transformer({'max_seq_length': 72, 'do_lower_case': False}) with Transformer model: BertModel 


In [3]:
# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용
# pooling model 출력 
print(pooling_model)
print(pooling_model.get_sentence_embedding_dimension())

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
768


In [4]:
# 3. dense 모델 추가(옵션)
#=> 필요에 따라 출력 dimension을 768보다 작게 줄이고 싶을때 dense 모델을 추가해서 줄임.
#=> https://www.sbert.net/docs/training/overview.html?highlight=dense 참조
if reduce_out_dimension:
    dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), # 입력 dimension은 앞에 pooling모델 embedding dimension으로 지정
                               out_features=out_dimension,  # 출력 dimension
                               activation_function=nn.Tanh())  # activation function은 Tahn으로 정의

In [5]:
# SBERT 모델 생성
if reduce_out_dimension:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
else:
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 72, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [7]:
# 훈련 데이터 불러오기
# => [sentence1, sentence2], labels 식으로 만듬
label2int = {"entailment": 0, "neutral": 1, "contradiction": 2}
train_samples = []

####################################################################################################
# KorNLI 훈련 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_kornli == 1:
    count = 0
    logger.info(f"\r\nRead NLI train dataset:{train_kornli_file}")

    with open(train_kornli_file, "rt", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            s1, s2, label = line.split('\t')
            label = label2int[label.strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")
            
            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
        
    logger.info(f'*kornli len: {count}')
####################################################################################################

####################################################################################################
# KlueNLI 훈련 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "nli", split="train")
# for data in datas:
#        s1 = data["sentence1"]
#        s2 = data["sentence2"]
#        label = data["label"]["label"]
###################################################################################################    
# kluenli 훈련인 경우 
if use_kluenli == 1:
    count = 0
    import json
    logger.info(f"\r\nRead NLI train dataset:{train_kluenli_file}")

    with open(train_kluenli_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            #print(data)
            s1 = data["premise"].strip()
            s2 = data["hypothesis"].strip()
            label = label2int[data["gold_label"].strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")

            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
            
    logger.info(f'*kluenli len: {count}')
    
####################################################################################################
# GLUENLI 훈련 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_gluenli == 1:
    count = 0
    logger.info(f"\r\nRead NLI train dataset:{train_gluenli_file}")

    with open(train_gluenli_file, "rt", encoding="utf-8") as f:
        lines = f.readlines()
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            s1, s2, label = line.split('\t')
            label = label2int[label.strip()]
            if count < 5:
                print(f"{s1}, {s2}, {label}")
            
            train_samples.append(InputExample(texts=[s1, s2], label=label))
            count += 1
        
    logger.info(f'*gluenli len: {count}')
####################################################################################################
        
logger.info(f'*train_samples_len:{len(train_samples)}')

2022-10-25 13:46:28,770 - s-bert - INFO - 
Read NLI train dataset:../../data11/korpora/gluemnli/glue-mnli-train.tsv


Conceptually cream skimming has two basic dimensions - product and geography., Product and geography are what make cream skimming work. , 1
you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him, You lose the things to the following level if the people recall., 0
One of our number will carry out your instructions minutely., A member of my team will execute your orders with immense precision., 0
How do you know? All this is their information again., This information belongs to them., 0
yeah i tell you what though if you go price some of those tennis shoes i can see why now you know they're getting up in the hundred dollar range, The tennis shoes have a range of prices., 1


2022-10-25 13:46:30,411 - s-bert - INFO - *gluenli len: 392702
2022-10-25 13:46:30,413 - s-bert - INFO - *train_samples_len:392702


In [8]:
# 데이터 셋, 데이터 로더, 손실함수 정의
train_dataset = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

# train_loss 정의 
# => 기본은 MSELoss 인데, NLI 훈련시에는 MultilpleNegativesRankingLoss(MNR Loss)가 좋다고 함.
train_loss = losses.MultipleNegativesRankingLoss(model)
#train_loss = losses.SoftmaxLoss(model=model, 
#                                sentence_embedding_dimension=model.get_sentence_embedding_dimension(), 
#                                num_labels=len(label2int))

In [10]:
#Read STSbenchmark dataset and use it as development set
# 평가데이터 불러오기
#korsts 파일로 두 문장간 유사도를 수치로(5.0이 만점=매우 유사) 측정함.
dev_samples = []

####################################################################################################
# KorSTS 평가 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_kornli == True:
    count = 0
    logger.info(f"\r\nRead STS dev dataset=>{eval_korsts_file}")
    with open(eval_korsts_file, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            text_a, text_b, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
            if count < 5:
                print(f"{text_a}, {text_b}, {score}")
            
            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
        logger.info(f'*korsts len: {count}')
####################################################################################################  

####################################################################################################
# KlueSTS 평가 데이터 셋 설정(.json 파일)
# => 아래처럼 load_dataset으로 불러와서 사용할수도 있음.
# datas = load_dataset("klue", "sts", split="test")
# for data in datas:
#        text_a = data["sentence1"]
#        text_b = data["sentence2"]
#        score = data["labels"]["label"]
#        score = float(score) / 5.0  
####################################################################################################           
if use_kluenli == True:
    count = 0
    logger.info(f"\r\nRead STS dev dataset=>{eval_kluests_file}")
    with open(eval_kluests_file, "rt", encoding="utf-8") as f:
        datas = json.load(f)
        for data in datas:
            text_a = data["sentence1"]
            text_b = data["sentence2"]
            score = data["labels"]["label"]
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함

            if count < 5:
                print(f"{text_a}, {text_b}, {score}")

            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
        logger.info(f'*kluests len: {count}')
####################################################################################################  

####################################################################################################
# GLUESTS 평가 데이터 셋 설정(.tsv 파일)
####################################################################################################
if use_gluenli == True:
    count = 0
    logger.info(f"\r\nRead STS dev dataset=>{eval_gluests_file}")
    with open(eval_gluests_file, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            text_a, text_b, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
            
            if count < 5:
                print(f"{text_a}, {text_b}, {score}")
            
            dev_samples.append(InputExample(texts= [text_a,text_b], label=score))
            count += 1
        logger.info(f'*gluests len: {count}')
####################################################################################################  
    
# 2개의 bert 모델에서 구한 2개의 embedding 값들의 cosine 유사도를 구해서, 이를 실제 score와 비교해서 유사도 측정함
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, 
                                                                 batch_size=eval_batch_size, 
                                                                 name='sts-dev')

2022-10-25 13:47:05,788 - s-bert - INFO - 
Read STS dev dataset=>../../data11/korpora/gluestsb/glue-stsb-valid.tsv
2022-10-25 13:47:05,833 - s-bert - INFO - *gluests len: 1500


A man with a hard hat is dancing., A man wearing a hard hat is dancing., 1.0
A young child is riding a horse., A child is riding a horse., 0.95
A man is feeding a mouse to a snake., The man is feeding a mouse to the snake., 1.0
A woman is playing the guitar., A man is playing guitar., 0.48000001907348633
A woman is playing the flute., A man is playing a flute., 0.55


In [11]:

warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up

# evaluation_steps은 20%로 설정
evaluation_steps = int(len(train_dataset) * num_epochs / eval_batch_size * 0.2)
logger.info(f"model:{model_path}, smodel:{smodel_path}")
logger.info("*train_batch: {}, eval_batch:{}, epoch:{}, lr:{}, eps:{}, train_dataset:{}, Warmup-steps: {}, evaluation_step: {}".format(train_batch_size, eval_batch_size, num_epochs, lr, eps, len(train_dataset), warmup_steps, evaluation_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          optimizer_params= {'lr': lr, 'eps': eps, 'correct_bias': False},
          save_best_model=True, # **기본 = True : eval 가장 best 모델을 output_Path에 저장함
          output_path=smodel_path
          )


2022-10-25 13:47:35,582 - s-bert - INFO - model:klue/bert-base, smodel:../../data11/model/NLI/sbert-klue-bert-base-gluenli
2022-10-25 13:47:35,585 - s-bert - INFO - *batch_size: 32, epoch:8, train_dataset:392702, Warmup-steps: 9818, evaluation_step: 9817
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12272 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.69 GiB total capacity; 2.51 GiB already allocated; 7.12 MiB free; 2.61 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
# => 훈련되어서 저장된 s-bert 모델을 불러와서 성능 평가 해봄
##############################################################################
import time

# 훈련완료후 테스트해볼 파일 경로(*Kluests_test 파일은 없어서, korsts test 파일 이용)
test_sts = '../../data11/korpora/korsts/tune_test.tsv'
test_batch_size = 32

# 테스트시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
test_output_path = "./output"
os.makedirs(test_output_path, exist_ok=True)

test_samples = []
with open(test_sts, 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        test_samples.append(InputExample(texts=[s1,s2], label=score))

logger.info("\n")
logger.info("======================TEST===================")
logger.info("\n\n")
logger.info(f"model save path > {smodel_path}")
start = time.time()
model = SentenceTransformer(smodel_path)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=test_batch_size, name='sts-test', show_progress_bar=True)
test_evaluator(model, output_path=test_output_path)
logger.info(f"처리시간 > {time.time() - start:.4f}")

2022-10-13 13:57:56,729 - s-bert - INFO - 

2022-10-13 13:57:56,732 - s-bert - INFO - ======================TEST===================
2022-10-13 13:57:56,732 - s-bert - INFO - 


2022-10-13 13:57:56,733 - s-bert - INFO - model save path > ../../data11/model/sbert/sbert-mdistilbertV3.1.1-disitl-ns-1.5


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

2022-10-13 13:57:59,538 - s-bert - INFO - 처리시간 > 2.8043


In [11]:
# 마지막 model 저장
#test_output_path = "../../data11/model/sbert/test"
#model.save(test_output_path)